# Brain Image Preprocessing Quick Start (Using DeepPrep)

**DeepPrep**: Deep learning-powered brain image preprocessing (Nature Methods 2025)
- 10× faster than fMRIPrep (~27 min vs 4-8 hours per subject)
- Processes both T1 (structural) and fMRI (functional) data
- Version: 25.1.0

## 1. Setup

In [ ]:
# Install required packages
!pip install nibabel pandas numpy matplotlib seaborn tqdm scikit-learn scipy

In [ ]:
import subprocess
from pathlib import Path
from typing import List, Optional
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

## 2. Configuration

In [ ]:
# Set your paths
BIDS_DIR = '/path/to/bids_data'
OUTPUT_DIR = '/path/to/output'
FS_LICENSE = '/path/to/freesurfer_license.txt'
DOCKER_IMAGE = 'pbfslab/deepprep:25.1.0'

# Subject IDs to process
SUBJECTS = ['001', '002', '003']

## 3. BIDS Data Preparation

In [ ]:
def create_bids_structure(base_dir, subject_id, t1_file=None, fmri_file=None):
    """Create BIDS directory for a subject"""
    import shutil
    import json
    
    sub_dir = Path(base_dir) / f'sub-{subject_id}'
    anat_dir = sub_dir / 'anat'
    func_dir = sub_dir / 'func'
    
    anat_dir.mkdir(parents=True, exist_ok=True)
    func_dir.mkdir(parents=True, exist_ok=True)
    
    if t1_file:
        dest = anat_dir / f'sub-{subject_id}_T1w.nii.gz'
        shutil.copy(t1_file, dest)
        print(f"✓ T1: {dest}")
    
    if fmri_file:
        dest = func_dir / f'sub-{subject_id}_task-rest_bold.nii.gz'
        shutil.copy(fmri_file, dest)
        print(f"✓ fMRI: {dest}")
    
    # Create dataset_description.json
    desc_file = Path(base_dir) / 'dataset_description.json'
    if not desc_file.exists():
        with open(desc_file, 'w') as f:
            json.dump({"Name": "My Dataset", "BIDSVersion": "1.6.0"}, f, indent=2)

# Example usage
# create_bids_structure(BIDS_DIR, '001', t1_file='/raw/t1.nii.gz', fmri_file='/raw/fmri.nii.gz')

## 4. DeepPrep Python Wrapper

In [ ]:
class DeepPrepRunner:
    def __init__(self, docker_image='pbfslab/deepprep:25.1.0', fs_license_path=None):
        self.docker_image = docker_image
        self.fs_license = Path(fs_license_path) if fs_license_path else None
        
        if self.fs_license and not self.fs_license.exists():
            raise FileNotFoundError(f"FreeSurfer license not found: {fs_license_path}")
    
    def run(self, bids_dir, output_dir, participant_labels=None, anat_only=False, 
            nthreads=4, device='cpu', skip_bids_validation=False):
        """Run DeepPrep preprocessing"""
        bids_dir = Path(bids_dir).resolve()
        output_dir = Path(output_dir).resolve()
        
        cmd = [
            'docker', 'run', '--rm', '-it',
            '-v', f'{bids_dir}:/input:ro',
            '-v', f'{output_dir}:/output',
        ]
        
        if self.fs_license:
            cmd.extend(['-v', f'{self.fs_license}:/license:ro'])
        
        if device == 'gpu':
            cmd.extend(['--gpus', 'all'])
        
        cmd.append(self.docker_image)
        cmd.extend(['/input', '/output', 'participant'])
        
        if self.fs_license:
            cmd.extend(['--fs-license-file', '/license'])
        
        cmd.extend(['--nthreads', str(nthreads), '--device', device])
        
        if participant_labels:
            cmd.extend(['--participant-label'] + participant_labels)
        
        if anat_only:
            cmd.append('--anat-only')
        
        if skip_bids_validation:
            cmd.append('--skip-bids-validation')
        
        print(f"Running: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✓ Success!")
        else:
            print(f"✗ Error: {result.stderr}")
        
        return result

## 5. Run DeepPrep

In [ ]:
# Initialize runner
runner = DeepPrepRunner(
    docker_image=DOCKER_IMAGE,
    fs_license_path=FS_LICENSE
)

# Process single subject
result = runner.run(
    bids_dir=BIDS_DIR,
    output_dir=OUTPUT_DIR,
    participant_labels=['001'],
    nthreads=8,
    device='cpu'
)

## 6. Data Loading

In [ ]:
class DeepPrepLoader:
    def __init__(self, output_dir, subject_id):
        self.output_dir = Path(output_dir) / 'deepprep' / f'sub-{subject_id}'
        self.subject_id = subject_id
    
    def load_t1(self, space='MNI152'):
        """Load preprocessed T1 image"""
        path = self.output_dir / 'anat' / f'sub-{self.subject_id}_space-{space}_T1w.nii.gz'
        img = nib.load(path)
        return img.get_fdata(), img.affine
    
    def load_bold(self, task='rest', space='MNI152'):
        """Load preprocessed BOLD image"""
        path = self.output_dir / 'func' / f'sub-{self.subject_id}_task-{task}_space-{space}_bold.nii.gz'
        img = nib.load(path)
        return img.get_fdata(), img.affine
    
    def load_confounds(self, task='rest'):
        """Load confounds for denoising"""
        path = self.output_dir / 'func' / f'sub-{self.subject_id}_task-{task}_confounds.tsv'
        return pd.read_csv(path, sep='\t')
    
    def load_tissue_masks(self):
        """Load GM, WM, CSF probability maps"""
        masks = {}
        for tissue in ['GM', 'WM', 'CSF']:
            path = self.output_dir / 'anat' / f'sub-{self.subject_id}_label-{tissue}_probseg.nii.gz'
            masks[tissue] = nib.load(path).get_fdata()
        return masks

In [ ]:
# Load data for subject 001
loader = DeepPrepLoader(OUTPUT_DIR, '001')

# Load T1
t1_data, t1_affine = loader.load_t1()
print(f"T1 shape: {t1_data.shape}")

# Load fMRI
bold_data, bold_affine = loader.load_bold()
print(f"BOLD shape: {bold_data.shape}")

# Load confounds
confounds = loader.load_confounds()
print(f"Confounds: {confounds.columns.tolist()}")

## 7. Quality Control

In [ ]:
def extract_qc_metrics(output_dir, subject_id, task='rest'):
    """Extract QC metrics from confounds"""
    loader = DeepPrepLoader(output_dir, subject_id)
    confounds = loader.load_confounds(task)
    
    fd = confounds['framewise_displacement'].values
    fd_mean = np.nanmean(fd)
    fd_max = np.nanmax(fd)
    
    dvars_mean = np.nanmean(confounds['dvars'].values) if 'dvars' in confounds.columns else None
    high_motion_tps = np.sum(fd > 0.5)
    
    return {
        'subject_id': subject_id,
        'fd_mean': fd_mean,
        'fd_max': fd_max,
        'dvars_mean': dvars_mean,
        'high_motion_tps': high_motion_tps,
        'total_tps': len(fd)
    }

# Run QC on all subjects
qc_results = [extract_qc_metrics(OUTPUT_DIR, s) for s in SUBJECTS]
qc_df = pd.DataFrame(qc_results)

# Filter by quality (FD < 0.5 mm)
good_subjects = qc_df[qc_df['fd_mean'] < 0.5]['subject_id'].tolist()
print(f"Subjects passing QC: {len(good_subjects)}/{len(SUBJECTS)}")
qc_df

In [ ]:
# Visualize QC metrics
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# FD distribution
axes[0].hist(qc_df['fd_mean'], bins=20, edgecolor='black')
axes[0].axvline(0.5, color='red', linestyle='--', label='Threshold')
axes[0].set_xlabel('Mean FD (mm)')
axes[0].set_ylabel('Count')
axes[0].set_title('Head Motion Distribution')
axes[0].legend()

# High-motion timepoints
axes[1].bar(range(len(qc_df)), qc_df['high_motion_tps'])
axes[1].set_xlabel('Subject Index')
axes[1].set_ylabel('High-Motion Timepoints')
axes[1].set_title('High-Motion TPs per Subject')

plt.tight_layout()
plt.show()

## 8. Denoising fMRI

In [ ]:
from sklearn.linear_model import LinearRegression

def denoise_bold(bold_data, confounds, confound_columns):
    """Remove confounds from BOLD data"""
    original_shape = bold_data.shape
    bold_2d = bold_data.reshape(-1, original_shape[-1]).T  # (time, voxels)
    
    X = confounds[confound_columns].fillna(0).values
    lr = LinearRegression()
    denoised = np.zeros_like(bold_2d)
    
    for i in range(bold_2d.shape[1]):
        y = bold_2d[:, i]
        lr.fit(X, y)
        denoised[:, i] = y - lr.predict(X)
    
    return denoised.T.reshape(original_shape)

# Denoise
confound_cols = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'csf', 'white_matter']
clean_bold = denoise_bold(bold_data, confounds, confound_cols)

print(f"Original BOLD shape: {bold_data.shape}")
print(f"Denoised BOLD shape: {clean_bold.shape}")

## 9. ROI Analysis & Functional Connectivity

In [ ]:
def extract_roi_timeseries(bold_data, atlas_data, roi_labels):
    """Extract mean timeseries for each ROI"""
    n_timepoints = bold_data.shape[-1]
    timeseries = np.zeros((len(roi_labels), n_timepoints))
    
    for i, label in enumerate(roi_labels):
        mask = atlas_data == label
        timeseries[i, :] = bold_data[mask].mean(axis=0)
    
    return timeseries

def compute_connectivity(timeseries, method='correlation'):
    """Compute functional connectivity matrix"""
    if method == 'correlation':
        return np.corrcoef(timeseries)
    elif method == 'partial_correlation':
        from sklearn.covariance import LedoitWolf
        estimator = LedoitWolf()
        estimator.fit(timeseries.T)
        precision = estimator.precision_
        d = np.sqrt(np.diag(precision))
        conn = -precision / np.outer(d, d)
        np.fill_diagonal(conn, 1)
        return conn

# Example: Load atlas and compute connectivity
# atlas_img = nib.load('atlas_MNI152.nii.gz')
# atlas_data = atlas_img.get_fdata()
# roi_ts = extract_roi_timeseries(clean_bold, atlas_data, range(1, 11))
# conn_matrix = compute_connectivity(roi_ts, method='correlation')

# # Visualize
# plt.figure(figsize=(8, 6))
# sns.heatmap(conn_matrix, cmap='coolwarm', center=0, vmin=-1, vmax=1)
# plt.title('Functional Connectivity Matrix')
# plt.show()

## 10. PyTorch Dataset for Deep Learning

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class BrainMRIDataset(Dataset):
    def __init__(self, output_dir, subject_ids, modality='T1', space='MNI152', transform=None):
        self.output_dir = Path(output_dir)
        self.subject_ids = subject_ids
        self.modality = modality
        self.space = space
        self.transform = transform
    
    def __len__(self):
        return len(self.subject_ids)
    
    def __getitem__(self, idx):
        subj_id = self.subject_ids[idx]
        loader = DeepPrepLoader(self.output_dir, subj_id)
        
        if self.modality == 'T1':
            data, _ = loader.load_t1(space=self.space)
        elif self.modality == 'BOLD':
            data, _ = loader.load_bold(space=self.space)
            data = np.mean(data, axis=-1)  # Average over time
        
        data = torch.FloatTensor(data).unsqueeze(0)  # Add channel dim
        data = (data - data.mean()) / (data.std() + 1e-8)  # Normalize
        
        if self.transform:
            data = self.transform(data)
        
        return data, subj_id

# Create dataset
dataset = BrainMRIDataset(OUTPUT_DIR, good_subjects, modality='T1')
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)

# Test
for batch_data, batch_ids in dataloader:
    print(f"Batch shape: {batch_data.shape}")
    print(f"Batch IDs: {batch_ids}")
    break

## 11. Complete Pipeline Function

In [ ]:
def preprocess_and_load(subject_ids, bids_dir, output_dir, fs_license, 
                       fd_threshold=0.5, nthreads=4):
    """
    Complete workflow: preprocess → QC → load data
    
    Returns:
        dict: Preprocessed data and QC metrics for passing subjects
    """
    # 1. Run DeepPrep
    print("Step 1: Running DeepPrep...")
    runner = DeepPrepRunner(fs_license_path=fs_license)
    
    for subj in subject_ids:
        result = runner.run(
            bids_dir=bids_dir,
            output_dir=output_dir,
            participant_labels=[subj],
            nthreads=nthreads
        )
        if result.returncode != 0:
            print(f"✗ Failed: {subj}")
    
    # 2. Quality control
    print("\nStep 2: Quality control...")
    qc_results = []
    for subj in subject_ids:
        try:
            metrics = extract_qc_metrics(output_dir, subj)
            qc_results.append(metrics)
        except Exception as e:
            print(f"QC failed for {subj}: {e}")
    
    qc_df = pd.DataFrame(qc_results)
    passing = qc_df[qc_df['fd_mean'] < fd_threshold]['subject_id'].tolist()
    print(f"QC passed: {len(passing)}/{len(subject_ids)}")
    
    # 3. Load data
    print("\nStep 3: Loading preprocessed data...")
    dataset = {}
    
    for subj in passing:
        try:
            loader = DeepPrepLoader(output_dir, subj)
            t1_data, affine = loader.load_t1()
            
            dataset[subj] = {
                'data': t1_data,
                'affine': affine,
                'qc': qc_df[qc_df['subject_id'] == subj].to_dict('records')[0]
            }
        except Exception as e:
            print(f"Failed to load {subj}: {e}")
    
    print(f"\nLoaded {len(dataset)} subjects")
    return dataset, qc_df

# Usage
# dataset, qc_df = preprocess_and_load(
#     subject_ids=['001', '002', '003'],
#     bids_dir=BIDS_DIR,
#     output_dir=OUTPUT_DIR,
#     fs_license=FS_LICENSE,
#     fd_threshold=0.5,
#     nthreads=8
# )

## Resources

- **Docs**: https://deepprep.readthedocs.io
- **GitHub**: https://github.com/pBFSLab/DeepPrep
- **Paper**: Nature Methods (2025) - https://www.nature.com/articles/s41592-025-02599-1
- **FreeSurfer License**: https://surfer.nmr.mgh.harvard.edu/registration.html (free)

**Learning**:
- BIDS: https://bids-specification.readthedocs.io
- Nilearn: https://nilearn.github.io
- MONAI: https://monai.io